In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import sys
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
from tasks import check

In [ ]:
from data import load_data
from lasagnekit.datasets.rescaled import Rescaled
from lasagnekit.easy import iterate_minibatches
from lasagnekit.datasets.imagecollection import ImageCollection
from lasagnekit.datasets.transformed import Transformed
from lasagnekit.datasets.stl import STL
from lasagnekit.datasets.helpers import load_once, split
from lasagnekit.datasets.subsampled import SubSampled
from lasagnekit.datasets.imagecollection import ImageCollection

w, h = 32, 32

folder = "{}/icons".format(os.getenv("DATA_PATH"))
data_scale1 = ImageCollection(size=(w, h), nb=1600, folder=folder, mode='all')
#data_scale1 = load_once(STL)(kind='unlabeled')
data_scale1.load()
del data_scale1.y

up = 2
scale = {1: w, 2: w/up}
batch_size = 128

c = 3



In [ ]:
data_scale1.X.shape

In [ ]:
nb_training_data = len(data_scale1.X)
valid_ratio = 0.1

In [ ]:
data_scale1.X = data_scale1.X[0:nb_training_data]

In [ ]:
data_scale1_train, data_scale1_test = split(data_scale1, test_size=valid_ratio)

In [ ]:
data_scale1_train = SubSampled(data_scale1, batch_size, mode='batch')
data_scale2_train = Rescaled(data_scale1_train, (scale[2], scale[2]))
data_scale2_train_up = Rescaled(data_scale2_train, (scale[1], scale[1]))

input_output_same_size = False
if input_output_same_size is True:
    data_scale2_train = data_scale2_train_up

In [ ]:
data_scale2_test = Rescaled(data_scale1_test, (scale[2], scale[2]))
data_scale2_test_up = Rescaled(data_scale2_test, (scale[1], scale[1]))
if input_output_same_size is True:
    data_scale2_test = data_scale2_test_up

In [ ]:
data_scale2_test.load()
data_scale2_test_up.load()

In [ ]:
def preprocess(x, s=32):
    x = x.reshape((x.shape[0], s, s, c))
    x = x.transpose((0, 3, 1, 2))
    return x

In [ ]:
from model import model58, model60, model62, model63

In [ ]:
filter_size = [2] + [5] * 4
all_layers = model63(nb_filters=128, filter_size=filter_size, w=None, h=None, c=c, up=up, nb_layers=3)

In [ ]:
import theano.tensor as T
import theano
from lasagne import layers

X = T.tensor4()
Y = T.tensor4()

Y_pred = (layers.get_output(all_layers['output'], X))
loss = ((Y_pred - Y) ** 2).mean()

In [ ]:
from lasagne import updates
import numpy as np
params = layers.get_all_params(all_layers.values(), trainable=True)
print("Number of params : {}".format(layers.count_params(all_layers.values(), trainable=True))) 
lr = theano.shared(np.array(0.1, dtype=theano.config.floatX))
params_updates = updates.adam(loss, params, learning_rate=lr)

In [ ]:
from lasagnekit.misc.draw_net import draw_to_file
from IPython.display import SVG
draw_to_file(layers.get_all_layers(all_layers['output']), 'out.svg')
SVG('out.svg')

In [ ]:
train_fn = theano.function([X, Y], loss, updates=params_updates, on_unused_input='warn')

In [ ]:
predict_fn = theano.function([X], Y_pred, on_unused_input='warn')

In [ ]:
from time import time, sleep
a = 0.001
lr.set_value(a)
train_loss = []
test_loss = []

In [ ]:
nb_epochs = 100
nb_updates_per_epoch = nb_training_data / batch_size
if nb_training_data % batch_size > 0:
    nb_updates_per_epoch += 1
nb_updates = nb_epochs * nb_updates_per_epoch

In [ ]:
start = time()
for epoch in range(nb_epochs):
    train_loss_cur = []
    for i in range(nb_updates_per_epoch):
        data_scale1_train.load()
        data_scale2_train.load()
        if input_output_same_size == False:
            data_scale2_train_up.load()
        # update parameters based on one minibatch
        X_scale1 = preprocess(data_scale1_train.X, s=scale[1])
        if input_output_same_size:
            X_scale2 = preprocess(data_scale2_train.X, s=scale[1])
        else:
            X_scale2 = preprocess(data_scale2_train.X, s=scale[2])
        X_scale2_up = preprocess(data_scale2_train_up.X, s=scale[1])
        loss_val = train_fn(X_scale2, X_scale1 - X_scale2_up)
        train_loss_cur.append(loss_val)
    total_loss = np.mean(train_loss_cur)
    print('Train loss : {}'.format(total_loss))
    train_loss.append(total_loss)
    #if epoch % 5 == 0 and i > 0:
    #    a = a / 2.
    #    lr.set_value(a)

    if epoch % 1 == 0:
        delta = time() - start
        start = time()
        print('Nb of seconds to the last validation step : {}'.format(delta))
        print("computing test performance")
        X_scale1 = preprocess(data_scale1_test.X, s=scale[1])
        if input_output_same_size:
            X_scale2 = preprocess(data_scale2_test.X, s=scale[1])
        else:
            X_scale2 = preprocess(data_scale2_test.X, s=scale[2])
        X_scale2_up = preprocess(data_scale2_test_up.X, s=scale[1])
        # test the model on test data
        total_loss = 0.
        nb = 0
        for mini_batch in iterate_minibatches(len(X_scale1), batch_size):
            resid = (X_scale1[mini_batch] - X_scale2_up[mini_batch])
            pred_resid = predict_fn(X_scale2[mini_batch])
            loss_val = ((pred_resid - resid) ** 2).mean()
            total_loss += loss_val
            nb += 1
        total_loss /= nb
        test_loss.append(total_loss)
        print('train_loss : {}, test_loss : {}'.format(train_loss[-1], test_loss[-1]))

In [ ]:
plt.plot(train_loss, label='train')
plt.plot(test_loss, label='test')
plt.legend()

In [ ]:
out1 = X_scale2[0:100]
for i in range(1):
    out1 = predict_fn(out1)
#out1 = X_scale1[0:100]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color

fig = plt.figure(figsize=(20, 20))
#fig = plt.figure()
img = dispims_color(X_scale2[0:100].transpose((0, 2, 3, 1)),border=1)
plt.imshow(img, interpolation='none')

fig = plt.figure(figsize=(20, 20))
#fig = plt.figure()

img = dispims_color((out1[0:100] + X_scale2_up[0:100]).transpose((0, 2, 3, 1)),border=1)
plt.imshow(img, interpolation='none')

fig = plt.figure(figsize=(20, 20))
#fig = plt.figure()

img = dispims_color(X_scale1[0:100].transpose((0, 2, 3, 1)),border=1)
plt.imshow(img, interpolation='none')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color

ss = np.random.uniform(size=(10, c, 32, 32)) 
ss = ss.astype(np.float32)
for i in range(1):
    ss = predict_fn(ss)
fig = plt.figure(figsize=(10, 10))
img = dispims_color(ss[0:100].transpose((0, 2, 3, 1)),border=1)
plt.axis('off')
plt.imshow(img, interpolation='none')

In [ ]:
outl = all_layers['output']
L = layers.get_all_layers(outl)
conv = L[1]
w = conv.W.get_value()
w = w.transpose((0, 1, 2, 3))
fig = plt.figure(figsize=(10, 10))
img = dispims_color(w.transpose((0, 2, 3, 1)), border=1)
plt.imshow(img, interpolation='none')

In [ ]:
from skimage.io import imread
from skimage.transform import rescale

D = imread('icon.png')
D = D[:, :, 0:3]
orig_D = D.copy()

D_rescaled = rescale(D, (1./2, 1./2), preserve_range=True) 
D_up = rescale(D_rescaled, (2, 2), preserve_range=True)

D_rescaled /= 255.
D_up /= 255.

D_rescaled = D_rescaled[None, :, :, :].transpose((0, 3, 1, 2))
D_up = D_up[None, :, :, :].transpose((0, 3, 1, 2))

D_rescaled = D_rescaled.astype(np.float32)
D_up = D_up.astype(np.float32)


Dhat = predict_fn(D_rescaled) + D_up

Dhat = Dhat.transpose((0, 2, 3, 1))
Dhat = Dhat[0]
Dhat = (Dhat - Dhat.min()) / (Dhat.max() - Dhat.min())
plt.imshow(D_rescaled.transpose((0, 2, 3, 1))[0], interpolation='none')
plt.show()
plt.imshow(Dhat, interpolation='none')
plt.show()
plt.imshow(orig_D, interpolation='none')
plt.show()